In [1]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from wrangle_xml import wrangle_xml, preprocessing_xml

In [3]:
df = wrangle_xml()

In [4]:
df.head(2)

,creationDate,device,endDate,sourceName,sourceVersion,startDate,type,unit,value
0,2016-06-03 17:38:46 -0500,"<<HKDevice: 0x2825a8410>, name:iPhone, manufac...",2016-06-03 16:40:52 -0500,Noah's iPhone,9.3.1,2016-06-03 16:40:09 -0500,HKQuantityTypeIdentifierStepCount,count,70.0
1,2016-06-03 17:38:46 -0500,"<<HKDevice: 0x2825a83c0>, name:iPhone, manufac...",2016-06-03 16:58:26 -0500,Noah's iPhone,9.3.1,2016-06-03 16:57:23 -0500,HKQuantityTypeIdentifierStepCount,count,68.0


In [5]:
df = preprocessing_xml(df)

In [6]:
df.head(2)

,creationDate,device,endDate,sourceName,sourceVersion,type,unit,value
startDate,,,,,,,,
2016-06-03 16:40:09,2016-06-03 17:38:46,"<<HKDevice: 0x2825a8410>, name:iPhone, manufac...",2016-06-03 16:40:52,Noah's iPhone,9.3.1,HKQuantityTypeIdentifierStepCount,count,70.0
2016-06-03 16:57:23,2016-06-03 17:38:46,"<<HKDevice: 0x2825a83c0>, name:iPhone, manufac...",2016-06-03 16:58:26,Noah's iPhone,9.3.1,HKQuantityTypeIdentifierStepCount,count,68.0


In [7]:
df_2 = pd.read_csv("Export.csv")

In [8]:
df_2['Date'] = pd.to_datetime(df_2.Date)

In [9]:
df_2.Date.agg([min, max])

min   2018-08-26
max   2020-08-16
Name: Date, dtype: datetime64[ns]

In [10]:
df_2.Date > "2020-01-01"

0       True
1       True
2       True
3       True
4       True
       ...  
584    False
585    False
586    False
587    False
588    False
Name: Date, Length: 589, dtype: bool

In [11]:
df.type.value_counts()

HKQuantityTypeIdentifierStepCount                 67915
HKQuantityTypeIdentifierDistanceWalkingRunning    67908
HKQuantityTypeIdentifierFlightsClimbed             1201
Name: type, dtype: int64

In [12]:
df = pd.read_csv("xml.csv")

In [13]:
df.type.value_counts()

HKQuantityTypeIdentifierStepCount                 67915
HKQuantityTypeIdentifierDistanceWalkingRunning    67908
HKQuantityTypeIdentifierFlightsClimbed             1201
Name: type, dtype: int64

In [24]:
df = wrangle_xml()

In [25]:
df_steps = df[df.type == 'HKQuantityTypeIdentifierStepCount']

In [26]:
df_steps = preprocessing_xml(df_steps)

/Users/noahsdisk/codeup-data-science/time_series_steps_project/wrangle_xml.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['endDate'] = df.endDate.str.replace("-0500", "")
/Users/noahsdisk/codeup-data-science/time_series_steps_project/wrangle_xml.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['startDate'] = df.startDate.str.replace("-0500", "")
/Users/noahsdisk/codeup-data-science/time_series_steps_project/wrangle_xml.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [28]:
df_steps = df_steps.reset_index()

In [29]:
create = df_steps.set_index('creationDate').resample("D").value.sum()

In [30]:
start = df_steps.set_index('startDate').resample("D").value.sum()

In [31]:
end = df_steps.set_index('endDate').resample("D").value.sum()

In [70]:
df_validate = pd.read_csv("Export.csv")

In [71]:
df_validate['Date'] = pd.to_datetime(df_validate.Date)

In [72]:
df_validate = df_validate.join(start, how='inner')

In [73]:
df_validate.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             0 non-null      datetime64[ns]
 1   Steps            0 non-null      int64         
 2   Distance         0 non-null      float64       
 3   Floors Ascended  0 non-null      int64         
 4   value            0 non-null      float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 0.0+ bytes


In [56]:
start = pd.DataFrame(start)
start.head()

,value
startDate,
2016-06-03,469.0
2016-06-04,4355.0
2016-06-05,4395.0
2016-06-06,6535.0
2016-06-07,6588.0


In [42]:
df_validate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589 entries, 0 to 588
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             589 non-null    object 
 1   Steps            589 non-null    int64  
 2   Distance         589 non-null    float64
 3   Floors Ascended  589 non-null    int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 18.5+ KB
